In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline

In [2]:
from google.colab import files 
  
  
uploaded = files.upload()

Saving full_tweets_df.csv to full_tweets_df.csv
Saving test_df.csv to test_df.csv
Saving train_df.csv to train_df.csv
Saving validate_df.csv to validate_df.csv


In [3]:
traindf=pd.read_csv('train_df.csv')
traindf.drop(['tweet_id', 'text_info_conf'],axis=1,inplace=True)
#tokenizer = Tokenizer(num_words=2000, split=' ')
testdf=pd.read_csv('test_df.csv')


In [4]:
traindf.head()
traindf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8933 entries, 0 to 8932
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   tweet_text  8933 non-null   object
 1   text_info   8933 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 139.7+ KB


In [5]:
X=traindf['tweet_text']
Y=traindf['text_info']
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

In [6]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)

In [7]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)


In [16]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(2*layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [19]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Model: "functional_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 150)]             0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 150, 50)           50000     
_________________________________________________________________
tf_op_layer_Mul_2 (TensorFlo [(None, 150, 50)]         0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 64)                29440     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation_6 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)              

In [20]:
model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Epoch 1/10
48/48 [==============================] - 1s 21ms/step - loss: 0.5440 - accuracy: 0.7279 - val_loss: 0.4433 - val_accuracy: 0.7992
Epoch 2/10
48/48 [==============================] - 1s 12ms/step - loss: 0.3905 - accuracy: 0.8271 - val_loss: 0.3971 - val_accuracy: 0.8269
Epoch 3/10
48/48 [==============================] - 1s 12ms/step - loss: 0.3522 - accuracy: 0.8472 - val_loss: 0.3960 - val_accuracy: 0.8295
Epoch 4/10
48/48 [==============================] - 1s 11ms/step - loss: 0.3315 - accuracy: 0.8566 - val_loss: 0.4060 - val_accuracy: 0.8249


In [21]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [22]:
accr = model.evaluate(test_sequences_matrix,Y_test)

42/42 [==============================] - 0s 5ms/step - loss: 0.4271 - accuracy: 0.8149


In [23]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Test set
  Loss: 0.427
  Accuracy: 0.815
